In [1]:
import pandas as pd

In [2]:
!gcloud

zsh:1: command not found: gcloud


In [ ]:
# pip install google-cloud-storage
# pip install pandas-gbq

In [5]:

from google.cloud import storage
from google.oauth2 import service_account
import json 
 
 # si ce code est ouvert dans cloud shell, plus besoin de fichier credentials
with open("ensai-2024-37c511689d57.json") as source:
    info = json.load(source)


credentials = service_account.Credentials.from_service_account_info(info)
prenoms = pd.read_gbq("ml.prenoms", project_id="ensai-2024", credentials=credentials)

In [6]:
prenoms.head()

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,13,37
1,1,_PRENOMS_RARES,1900,57,37
2,1,_PRENOMS_RARES,1900,59,44
3,1,_PRENOMS_RARES,1900,62,34
4,1,_PRENOMS_RARES,1900,75,45


### Models

In [8]:
len(set(prenoms["preusuel"]))

36171

### Preprocessing

In [9]:
import sklearn 
from sklearn.model_selection import train_test_split

preprocessing = prenoms[prenoms["preusuel"] != '_PRENOMS_RARES']
preprocessing

,sexe,preusuel,annais,dpt,nombre
159,1,ABEL,1901,59,34
160,1,ABEL,1902,62,35
161,1,ABEL,1903,62,33
162,1,ABEL,1905,59,39
163,1,ABEL,1905,62,39
...,...,...,...,...,...
3784668,2,ZELIMA,XXXX,XX,30
3784669,2,ZENABA,XXXX,XX,30
3784670,2,ZEREN,XXXX,XX,30
3784671,2,ZINABE,XXXX,XX,30


In [25]:
preprocessing = preprocessing.groupby(["preusuel", "sexe"]).agg({"nombre": sum}).reset_index()

In [30]:
data = preprocessing.iloc[preprocessing.groupby('preusuel')['nombre'].idxmax()]

TypeError: reduction operation 'argmax' not allowed for this dtype

In [27]:
preprocessing.idxmax()

TypeError: reduction operation 'argmax' not allowed for this dtype

In [12]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import string

# Créer un dictionnaire associant chaque lettre de l'alphabet à un indice unique
alphabet_indices = {letter: index for index, letter in enumerate(string.ascii_lowercase)}

# Fonction pour convertir un prénom en vecteur one-hot basé sur l'apparence des lettres
def name_to_one_hot(name):
    vector = [0] * len(alphabet_indices)
    for letter in name.lower():
        if letter in alphabet_indices:
            vector[alphabet_indices[letter]] = 1
    return vector

# Liste de prénoms (exemple)
names = ['Alice-Na', 'Bob', 'Charlie']

# Transformer chaque prénom en vecteur one-hot
one_hot_encoded_names = [name_to_one_hot(name) for name in names]

# Afficher les prénoms one-hot encoded
print("Prénoms one-hot encoded :")
for name, one_hot_vector in zip(names, one_hot_encoded_names):
    print(name + ":", one_hot_vector)

Prénoms one-hot encoded :
Alice-Na: [1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Bob: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Charlie: [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
